In [2]:
import pyarrow as pa
import ctypes
import arrow_hasher as ah

def hash_arrow_table(table: pa.Table):
    # Covert table to record batch first (so we can extract the pointers), since the default behavior is 1 batch, we can just get the first element
    # After that we can extract the PyCapsules
    schema_capsule, array_capsule = table.to_batches()[0].__arrow_c_array__()

    # Extract raw pointers from capsules due to uniffi limitations
    PyCapsule_GetPointer = ctypes.pythonapi.PyCapsule_GetPointer
    PyCapsule_GetPointer.argtypes = [ctypes.py_object, ctypes.c_char_p]
    PyCapsule_GetPointer.restype = ctypes.c_void_p

    return ah.process_arrow_table(PyCapsule_GetPointer(array_capsule, b"arrow_array"),  PyCapsule_GetPointer(schema_capsule, b"arrow_schema"))

# Create a simple Arrow table
data = {
    'id': [1, 2, 3, 4, 5],
    'name': ['Alice', 'Bob', 'Charlie', 'David', 'Eve'],
    'value': [10.5, 20.3, 15.7, 30.2, 25.8]
}
table = pa.table(data)

hash_arrow_table(table)



Received table with 5 rows and 3 columns
Table SHA-256 hash: e474f034a7d25abfac4941a3239a3d7c56405c84edb866e474056cbe033a9476


'e474f034a7d25abfac4941a3239a3d7c56405c84edb866e474056cbe033a9476'